## Import Necessary Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

import logging
logging.getLogger("distributed.worker.memory").setLevel(logging.CRITICAL)
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit='4GB'
)
client = Client(cluster)


import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Loading Data

In [5]:
import os
print(os.getcwd())

/media/sheikh/F262ADC762AD90C1/backup/ML/yellow-taxi-demand-analysis/1_Data_Preprocess/a_EDA


In [6]:
ddf = dd.read_parquet("../../3_Data/raw/yellow_tripdata_2025-01_(january).parquet", npartitions=1)
ddf.compute().head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0


In [7]:
ddf.compute().head().T

,0,1,2,3,4
VendorID,1,1,1,2,2
tpep_pickup_datetime,2025-01-01 00:18:38,2025-01-01 00:32:40,2025-01-01 00:44:04,2025-01-01 00:14:27,2025-01-01 00:21:34
tpep_dropoff_datetime,2025-01-01 00:26:59,2025-01-01 00:35:13,2025-01-01 00:46:01,2025-01-01 00:20:01,2025-01-01 00:25:06
passenger_count,1.0,1.0,1.0,3.0,3.0
trip_distance,1.6,0.5,0.6,0.52,0.66
RatecodeID,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N
PULocationID,229,236,141,244,244
DOLocationID,237,237,141,244,116
payment_type,1,1,1,2,2


In [8]:
# let's set "tpep_pickup_datetime" as index

In [9]:
# Convert to datetime if not already
ddf['tpep_pickup_datetime'] = dd.to_datetime(ddf['tpep_pickup_datetime'])

# Set as index
ddf = ddf.set_index('tpep_pickup_datetime', sorted=True)

# Persist (optional, helps for faster future operations)
ddf = ddf.persist()

ddf.compute().head().T

tpep_pickup_datetime,2025-01-01 00:18:38,2025-01-01 00:32:40,2025-01-01 00:44:04,2025-01-01 00:14:27,2025-01-01 00:21:34
VendorID,1,1,1,2,2
tpep_dropoff_datetime,2025-01-01 00:26:59,2025-01-01 00:35:13,2025-01-01 00:46:01,2025-01-01 00:20:01,2025-01-01 00:25:06
passenger_count,1.0,1.0,1.0,3.0,3.0
trip_distance,1.6,0.5,0.6,0.52,0.66
RatecodeID,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N
PULocationID,229,236,141,244,244
DOLocationID,237,237,141,244,116
payment_type,1,1,1,2,2
fare_amount,10.0,5.1,5.1,7.2,5.8


In [10]:
ddf.compute().tail()

,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,
2025-01-31 23:01:48,2,2025-01-31 23:16:29,NaN,3.35,NaN,<NA>,79,237,0,15.85,0.0,0.5,0.0,0.0,1.0,20.60,NaN,NaN,0.75
2025-01-31 23:50:29,2,2025-02-01 00:17:27,NaN,8.73,NaN,<NA>,161,116,0,28.14,0.0,0.5,0.0,0.0,1.0,32.89,NaN,NaN,0.75
2025-01-31 23:26:59,2,2025-01-31 23:43:01,NaN,2.64,NaN,<NA>,144,246,0,14.91,0.0,0.5,0.0,0.0,1.0,19.66,NaN,NaN,0.75
2025-01-31 23:14:34,2,2025-01-31 23:34:52,NaN,3.16,NaN,<NA>,142,107,0,17.55,0.0,0.5,0.0,0.0,1.0,22.30,NaN,NaN,0.75
2025-01-31 23:56:42,2,2025-02-01 00:07:27,NaN,2.29,NaN,<NA>,237,238,0,12.09,0.0,0.5,0.0,0.0,1.0,16.09,NaN,NaN,0.00


In [11]:
ddf.compute().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3475226 entries, 2025-01-01 00:18:38 to 2025-01-31 23:56:42
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_dropoff_datetime  datetime64[us]
 2   passenger_count        float64       
 3   trip_distance          float64       
 4   RatecodeID             float64       
 5   store_and_fwd_flag     string        
 6   PULocationID           int32         
 7   DOLocationID           int32         
 8   payment_type           int64         
 9   fare_amount            float64       
 10  extra                  float64       
 11  mta_tax                float64       
 12  tip_amount             float64       
 13  tolls_amount           float64       
 14  improvement_surcharge  float64       
 15  total_amount           float64       
 16  congestion_surcharge   float64       
 17  Airport_fee            float64  

In [12]:
# There is dtypes: datetime64[us](1), float64(13), int32(3), int64(1), string(1)
# 3475226 rows, 19 columns

In [13]:
ddf.compute().describe().T

,count,mean,min,25%,50%,75%,max,std
VendorID,3475226.0,1.785428,1.0,2.0,2.0,2.0,7.0,0.426328
tpep_dropoff_datetime,3475226,2025-01-17 11:17:56.997901,2024-12-18 07:52:40,2025-01-10 08:15:29.500000,2025-01-17 15:59:34,2025-01-24 19:48:31,2025-02-01 23:44:11,NaN
passenger_count,2935077.0,1.297859,0.0,1.0,1.0,1.0,9.0,0.75075
trip_distance,3475226.0,5.855126,0.0,0.98,1.67,3.1,276423.57,564.6016
RatecodeID,2935077.0,2.482535,1.0,1.0,1.0,1.0,99.0,11.632772
PULocationID,3475226.0,165.191576,1.0,132.0,162.0,234.0,265.0,64.529483
DOLocationID,3475226.0,164.125177,1.0,113.0,162.0,234.0,265.0,69.401686
payment_type,3475226.0,1.036623,0.0,1.0,1.0,1.0,5.0,0.701333
fare_amount,3475226.0,17.081803,-900.0,8.6,12.11,19.5,863372.12,463.472918
extra,3475226.0,1.317737,-7.5,0.0,0.0,2.5,15.0,1.861509


In [14]:
ddf.compute().columns

Index(['VendorID', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'Airport_fee', 'cbd_congestion_fee'],
      dtype='object')

In [15]:
# We just need 'VendorID','tpep_pickup_datetime','passenger_count' this three columns for this project


In [16]:
# Check dataset shape and columns
print("Rows:", len(ddf))
print("Columns:", len(ddf.columns))
print("Columns_Name:", list(ddf.columns))

Rows: 3475226
Columns: 19
Columns_Name: ['VendorID', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee', 'cbd_congestion_fee']


In [18]:
#Trip duration (in minutes)
df = ddf.compute()
df['trip_duration_min'] = (
    pd.to_datetime(df['tpep_dropoff_datetime']) -
    df.index.to_series()
).dt.total_seconds() / 60

df[['trip_distance', 'trip_duration_min', 'total_amount']].head()


,trip_distance,trip_duration_min,total_amount
tpep_pickup_datetime,,,
2025-01-01 00:18:38,1.60,8.350000,18.00
2025-01-01 00:32:40,0.50,2.550000,12.12
2025-01-01 00:44:04,0.60,1.950000,12.10
2025-01-01 00:14:27,0.52,5.566667,9.70
2025-01-01 00:21:34,0.66,3.533333,8.30
